In [27]:
%%writefile mapper1.py

# Your code for mapper here.
import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        if ((word[1:].islower()) and (word[0].isupper()) and (not word[0].isdigit())):
            print "%s\t%d\t%d" % (word.lower(), 1, 1)
        else:
            print "%s\t%d\t%d" % (word.lower(), 0, 1)

Overwriting mapper1.py


In [28]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_key = None
word_name_sum = 0
word_total_sum = 0

for line in sys.stdin:
    try:
        key, count_name, count_total = line.strip().split('\t', 2)
        count_name = int(count_name)
        count_total = int(count_total)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print "%s\t%d\t%d" % (current_key, word_name_sum, word_total_sum)
        word_name_sum = 0
        word_total_sum = 0
        current_key = key
    word_name_sum += count_name
    word_total_sum += count_total

if current_key:
    print "%s\t%d\t%d" % (current_key, word_name_sum, word_total_sum)

Overwriting reducer1.py


In [29]:
%%writefile mapper2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        word, count_name, count_total = line.strip().split('\t', 2)
        count_name = int(count_name)
        count_total = int(count_total)
        if float(count_name)/count_total > 0.995:
            print '%d\t%s' % (count_name, word)
    except ValueError as e:
        continue

Overwriting mapper2.py


In [30]:
%%writefile reducer2.py

import sys
reload(sys)

for line in sys.stdin:
    try:
        count, word = line.strip().split('\t', 1)
        count = int(count)
        print '%s\t%d' % (word, count)
    except ValueError as e:
        continue

Overwriting reducer2.py


In [31]:
%%bash

OUT_DIR="assignment1_wordcount_"$(date +"%s%6N")

NUM_REDUCERS=8

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null

OUT_DIR1="assignment1_sorting_"$(date +"%s%6N")

NUM_REDUCERS=1

hdfs dfs -rm -r -skipTrash ${OUT_DIR1} > /dev/null

# Code for your second job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming sorting" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.partition.keycomparator.options="-nr" \
    -files mapper2.py,reducer2.py \
    -mapper "python mapper2.py" \
    -reducer "python reducer2.py" \
    -input ${OUT_DIR} \
    -output ${OUT_DIR1} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR1}/part-00000 | sed -n 5p
hdfs dfs -rm -r -skipTrash ${OUT_DIR1}* > /dev/null

french	5741


rm: `assignment1_wordcount_1534517230823602': No such file or directory
18/08/17 14:47:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/17 14:47:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/08/17 14:47:14 INFO mapred.FileInputFormat: Total input files to process : 1
18/08/17 14:47:14 INFO mapreduce.JobSubmitter: number of splits:2
18/08/17 14:47:15 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1534512728888_0013
18/08/17 14:47:15 INFO impl.YarnClientImpl: Submitted application application_1534512728888_0013
18/08/17 14:47:15 INFO mapreduce.Job: The url to track the job: http://d1967598ec2c:8088/proxy/application_1534512728888_0013/
18/08/17 14:47:15 INFO mapreduce.Job: Running job: job_1534512728888_0013
18/08/17 14:47:20 INFO mapreduce.Job: Job job_1534512728888_0013 running in uber mode : false
18/08/17 14:47:20 INFO mapreduce.Job:  map 0% reduce 0%
18/08/17 14:47:36 INFO mapreduce.Job:  map 24% reduce 0%
18/08/17